# Model for multiple compounds diffusion, sorption and reaction

The code requires:
-	To first create a folder which will collect all the files corresponding the simulation
- In the folder, make sure you have the different software available
-	To create input files using the command available in FloPy
-	To run the mf (modflow), mt (mt3dms) or sw (seawat) software (through FloPy)
-	For PHT3D: verify the ontant of the _datab.DAT and _ph.DAT files
-	To run PHT3D in the same folder, using cmd or Python


In [ ]:
# Get libraries
import numpy as np
import flopy
# Get librairies for post processing the results
import matplotlib as plt
import flopy.utils.binaryfile as bf
import os
# Assign name and create modflow model object
modelname = 'Model_multi_comp'
mfexe = 'C:\\Program Files (x86)\\PM8\\modflow2005\\mf2005'
mtexe = 'C:\\mt3d-usgs_1.0\\mt3d-usgs_Distribution\\bin\\MT3D-USGS_64'
#mtexe = 'C:\\Program Files (x86)\\PM8\\mt3d\\mt3di'
#modelpth = 'D:\0_Recent_files\Flopy\Model3draft'                                         #
path_pht3d='D:\\0_Recent_files\\PHT3d\\0_Model_flopy\\' + modelname
mf = flopy.modflow.Modflow(modelname, exe_name= mfexe)#, model_ws=workspace)


In [ ]:
# Set model: clay layer between two aquifers
# Note: we turn the model around, therefore the columns are along the longitudinal axis, the row along the vertical axis and the layers are along the lateral axis 
# 2 m thick plume above the clay layer
Lx = 100.   # m Model length
Ly = 10.    # m Model thickness
ztop = 0.   # m 
zbot = -50.
nlay = 1    # Number layers
nrow = 100  # Number row
ncol = 100  # Number col
delr = Lx/ncol # Discretization (confusing use of "r" in delr as in "row")
delc = Ly/nrow # Discretization (confusing use of "c" in delc as in "column")
delv = (ztop - zbot) / nlay
botm = np.linspace(ztop, zbot, nlay + 1)

# Model 
# Thickness layers in meters
th_y01 = 1 # m Bottom sand layer thickness
th_y12 = 1 # m Clay layer thickness
th_y23 = Ly - (th_y01+th_y12) # m Upper sand layer thickness
nrowtop = int(th_y23/delc)
nrowbott =  int((th_y12+th_y23)/delc)
#PLUME
th_pl = 2 #m thickness plume
#This model implies the plume bottom is at the surface of the aquitard
nrow_pl_top  = int((th_y23 - th_pl)/delc)
nrow_pl_bott = int((th_y23 )/delc)

# MODFLOW
Vx_y = 36.5 # m/y velocity unit per year
K_cl = 1. # per year
K_sd = 3650. # per year
rhob_sd = 1.855
prsity_sd = 0.35
al_cl = 0.01 # NOT YET USED
al_sd = 1. 
al = np.ones((nlay, nrow, ncol)) # no differentiation clay and sand for longitudinal disp
trpt = 0.0015
trpv = 0.0015
dmcoef = 0.05  #m2/y

# 
# HK array for LPF package
hk = np.ones((nlay, nrow, ncol), dtype=np.int32)
hk = hk*K_sd
hk[:, nrowtop:nrowbott, :] = K_cl
# Longitudinal disp
al = np.ones((nlay, nrow, ncol))
al = al*al_sd
#al[:, nrowtop:nrowbott, :] = al_cl

# Variables for the BAS package; set the hydraulic gradient
ibound = np.ones((nlay, nrow, ncol))#, dtype=np.int32)
ibound[:, :, 0] = -1
ibound[:, :, -1] = -1
# Hydraulic gradient
hg = (prsity_sd*Vx_y*Lx)/(K_sd)  # m head
strt = np.ones((nlay, nrow, ncol))#, dtype=np.float32)
strt = 10.*strt
strt[:, :, 0] = 10.
strt[:, :, -1] = 10. - hg  

# TIME
perlen = 20 # lentgh stress period
nstp = 40 # number time steps / stress period
time_unit_year = 5 # itmuni {0: "undefined", 1: "seconds", 2: "minutes",3: "hours", 4: "days", 5: "years"}

In [ ]:
# Multi component 
# Initial concentration
# careful species numbers 1 to n, python numbering 0 to n-1
C1 = 0.9778992## 7 digits (?) more seem not to work, sum C1 + C2 + C3 = 1
C2 = 0.0219773#
C3 = 0.0001234#
C0 = 0.
df1 = 0.05288659 # Diff coef # PB IC CANNOT CHECK BECAUSE TIME UNIT ISSUE
df2 = 0.05287496
df3 = 0.05286343
Kd1 = 2.5
Kd2 = 2.5
Kd3 = 2.5


MODFLOW

In [ ]:
# Creates the MODFLOW files

# Create the discretization object
# perlen = stress period duration
# ntsp = number time steps
# itmuni = time unit 1 = sec; 4 = day;
# Steady = whether the stress period is steady state False/True 
dis = flopy.modflow.ModflowDis(mf, nlay, nrow, ncol, 
                               delr=delr, delc=delc,
                               top=ztop, botm=botm[1:],
                               perlen = perlen, nstp = nstp,
                               itmuni = time_unit_year)#, steady = False)

# BAS Basic package class # ibound : constant head # strt : start head
bas = flopy.modflow.ModflowBas(mf, ibound=ibound, strt=strt)
# LPF package to the MODFLOW model 
# hk hydraulic conductivity 
# vka: vertical hydraulic conductivity
lpf = flopy.modflow.ModflowLpf(mf, hk=hk) #vka=10.

# OC package to the MODFLOW model # Output Control
# example uses #spd = {(0, 0): ['print head', 'print budget', 'save head', 'save budget']}
oc = flopy.modflow.ModflowOc(mf)#, stress_period_data=spd, compact=True)

# PCG package # PCG stands for??
pcg = flopy.modflow.ModflowPcg(mf)

pth = os.getcwd()
print(pth)

# GET MT3DMS package to run
# Add LMT Link-MT3DMS package to the MODFLOW model #produces ftl file, ftl =  Flow Transport Link 
lmt = flopy.modflow.mflmt.ModflowLmt(mf)
#####
# Write the MODFLOW model input files
mf.write_input()
# Run the MODFLOW model
mf.run_model()

MT3DMS model

In [ ]:
### MT3DMS model starts

# Calls the modflow model with "MODFLOWMODEL"
mt = flopy.mt3d.Mt3dms(modelname, modflowmodel = mf,  exe_name= mtexe, version='mt3d-usgs',ftlfilename= 'mt3d_link.ftl', ftlfree=True )# model_ws=workspace)#ftlfilename= 'mt3d_link.ftl' ,modflowmodel = mf,
#mt = flopy.mt3d.Mt3dms(modelname,  exe_name= mtexe, model_ws=workspace)#ftlfilename= 'mt3d_link.ftl' ,
print(mt.namefile)
### Add ADV package to the MT3DMS model # Advection
adv = flopy.mt3d.Mt3dAdv(mt)

### BTN PACKAGE # Basic transport
#Set cells with fixed concentration
icbund = np.ones((nlay, nrow, ncol))#, dtype=np.int32)
# Plume position (above aquitard)
icbund[:, nrow_pl_top:nrow_pl_bott, 0] = -1
# Set initial concentrations
sconc = np.zeros((nlay, nrow, ncol))#, dtype=np.int32) # concentrations set with stress periods
# Plume position (above aquitard)
#sconc = np.ones((nlay, nrow, ncol))*C0
#sconc[:, nrow_pl_top:nrow_pl_bott, 0] = C0

# BTN package
# ncomp: number of species
# mcomp : number of mobile species
# sconc : starting conc, icbund= fixed conc
# prsity:  porosity
# ifmtcn : int MAYBE IMPORTANT ???
# species_names = list of str
# obs = array with cells indices (lay,row,col) for which conc has to be printed
# nprobs = how frequently the concentration should be saved
btn = flopy.mt3d.Mt3dBtn(mt, ncomp = 3, mcomp = 3,
                         icbund = icbund, sconc = sconc, 
                         prsity = prsity_sd)#,species_names = ['SP0', 'SP1'])#  mcomp = 1,  # nprobs = 10, # species_names = ['SP1'])

### DSP PACKAGE
# Dispersion
# al longitudinal disp
# trpt: ratios transverse horizontal/longitudinal
# trpv : ratio tranverse vertical/longitudinal
# dmcoef: effective molecular diffusion coefficient
# try MULTI DIFF when normal setting work
dsp = flopy.mt3d.Mt3dDsp(mt,  al = al_sd, 
                         trpt = trpt, trpv = trpv,
                         dmcoef = df1, multiDiff= True, dmcoef2 = df2, dmcoef3 = df3)
###RCT PACKAGE
# Reaction
# isothm = 1 # linear sorption
# ireact # type of kinetic rate
# igetsc # taking into account initial sorbed conc
# rhob # bulk density
# prsity2 # porosity immobile domain (not mobile)
# sp1 soprtion paramters, for isothm = 1 sp1 = Kd sp1 float or array
rct = flopy.mt3d.Mt3dRct(mt,
                         isothm = 1,
                         ireact = 0,
                         igetsc = 0,
                         rhob = rhob_sd,
                         sp1 = Kd1, sp12 =Kd2, sp13 =Kd3)

#TOB package # Transport observation package
#tob = flopy.mt3d.Mt3dTob(mt)

# SSM PACKAGE # Source sink Mix package
# crch: concentration recharge of specie 1. If the conc change/stres period, provide a dictionary
# stress_period_data : dictionary
itype = flopy.mt3d.Mt3dSsm.itype_dict()
ssm_data = {}
nr_idx = np.arange(nrow_pl_top , nrow_pl_bott)
# ssm_data[0]: first stress period / (lay,col,row,concentration initial, itype['CC']) 
#itype['CC'] = -1 type = constant concentration
# Additional info for multi components: add concentrations for each compound after itype
ssm_data[0] = [[0,nr,0,C0,itype['CC'],C1,C2,C3] for nr in nr_idx]
ssm = flopy.mt3d.Mt3dSsm(mt, stress_period_data=ssm_data)#, unitnumber) #stress_period_data=ssm_data

# Gcg Generalized conjugate gradient
gcg = flopy.mt3d.Mt3dGcg(mt)                      

#pth = os.getcwd() not necessary

# Write the MT3DMS model input files
mt.write_input()

# Run the MT3DMS model
#####  mt.run_model()  (not necessary if you run PHT3D)



In [ ]:
# Post-process the results
# Load data
ucnobj = flopy.utils.binaryfile.UcnFile('MT3D001.UCN')
times = ucnobj.get_times()
# print times
concentration = ucnobj.get_data(totim=times[-1])

# Make the plot
# import matplotlib.pyplot as plt
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(1, 1, 1, aspect='equal')
ax.imshow(concentration[0, :, :], interpolation='nearest',
          extent=(0, Lx, 0, Ly))
ax.set_title('Concentrations')
plt.savefig(os.path.join(path_pht3d +'.png'))
plt.show()


In [ ]:
# Get heads
hds = bf.HeadFile(modelname+'.hds')#, text='drawdown', precision='single')
times = hds.get_times()
head = hds.get_data(totim=times[-1])
print('Head statistics')
print('  min: ', head.min())
print('  max: ', head.max())
print('  std: ', head.std())


In [ ]:
# Fill in parameters faster
# Create text files as below, input values for a maximum of flexibility (except temp is fixed to 10C for diffusion)
"""
#inputs_model_full.txt  # PCE not included
index Koc Mav_0_permil diffav_0_pm_10
TCE 142.56075936021887 131.38822426592381 5.8695520505754194e-10
DCE 74.81695005111546 96.943424265923809 6.8958077384620149e-10
VC 39.26449353995999 62.498624265923809 8.7021893843665972e-10
ETH 26.668586645214805 28.053824265923815 1.3304669012863884e-09
Tracer 142.56075936021887 131.38822426592381 5.8695520505754194e-10

#MW_abund_isotopologue.txt  # PCE included
Index MW Abundance_0_pm Element CE Isotopologues
Pce_c_ll 165.8108 0.977899 C pce Pce_c_ll
Pce_c_lh 166.8108 0.021977 C pce Pce_c_lh
Pce_c_hh 167.8108 0.000123 C pce Pce_c_hh
Tce_c_ll 131.3660 0.977899 C tce Tce_c_ll
Tce_c_lh 132.3660 0.021977 C tce Tce_c_lh
Tce_c_hh 133.3660 0.000123 C tce Tce_c_hh
Dce_c_ll 96.9212 0.977899 C dce Dce_c_ll
Dce_c_lh 97.9212 0.021977 C dce Dce_c_lh
Dce_c_hh 98.9212 0.000123 C dce Dce_c_hh
Vc_c_ll 62.4764 0.977899 C vc Vc_c_ll
Vc_c_lh 63.4764 0.021977 C vc Vc_c_lh
Vc_c_hh 64.4764 0.000123 C vc Vc_c_hh
Vc_cl_l 62.0457 0.757710 Cl vc Vc_cl_l
Vc_cl_h 64.0457 0.242290 Cl vc Vc_cl_h
Pce_llll 164.0220 0.329619 Cl pce Pce_llll
Pce_hhhh 172.0220 0.003446 Cl pce Pce_hhhh
Pce_lhhh 170.0220 0.043109 Cl pce Pce_lhhh
Pce_llhh 168.0220 0.202222 Cl pce Pce_llhh
Pce_lllh 166.0220 0.421604 Cl pce Pce_lllh
Tce_lll 130.0299 0.435020 Cl tce Tce_lll
Tce_hhh 136.0299 0.014223 Cl tce Tce_hhh
Tce_llh 132.0299 0.139105 Cl tce Tce_llh
Tce_lhl 132.0299 0.139105 Cl tce Tce_lhl
Tce_hll 132.0299 0.139105 Cl tce Tce_hll
Tce_lhh 134.0299 0.044481 Cl tce Tce_lhh
Tce_hlh 134.0299 0.044481 Cl tce Tce_hlh
Tce_hhl 134.0299 0.044481 Cl tce Tce_hhl
Dce_ll 96.0378 0.574125 Cl dce Dce_ll
Dce_lh 98.0378 0.367171 Cl dce Dce_lh
Dce_hh 100.0378 0.058704 Cl dce Dce_hh
Cl_l 35.0000 0.757710 Cl cl Cl_l
Cl_h 37.0000 0.242290 Cl cl Cl_h
Eth_c_ll 28.0316 0.977899 C eth Eth_c_ll
Eth_c_lh 29.0316 0.021977 C eth Eth_c_lh
Eth_c_hh 30.0316 0.000123 C eth Eth_c_hh

"""
inputs = pd.read_csv(path_plot + '\\inputs_model_full.txt',delimiter = ' ', index_col = 0 , header = 0)
MW_abund_isotopologue = pd.read_csv(path_plot + '\\MW_abund_isotopologue.txt',delimiter = ' ', index_col = 0 , header = 0)#xls.parse(worksheetname, index_col=0, na_values=['NaN'], usecols = [0,1,2], header = 1)

temp = 10 # (as in the text file input_model_full
Mav_0_permil =   inputs['Mav_0_permil']   # Mass at 0 permil - average [131.38822426592381, 96.943424265923809, 62.498624265923809, 28.053824265923815]
diffav_0_pm_10 = inputs['diffav_0_pm_10'] # import Diffusion coefficient at 10 C per sec #[5.8695520505754194e-10, 6.8958077384620149e-10, 8.7021893843665972e-10, 1.3304669012863884e-09]
Koc =            inputs['Koc']            # import Koc #[142.56075936021887, 74.81695005111546, 39.26449353995999, 26.668586645214805] #logKow = [2.4,1.9,1.4,1.1] xxxADD EQUATION# for info

# I created functions for calculating the koc for each compound
"""def set_koc(Name_ce_group,alpha,Koc_ce_ave):
    # set_koc provides the respective Koc for each isotopologues
    # inputs are the CE group (carbon or chlorine isotope), the factionation factors and the avecrage KOC for each CE
    Koc_iso = []
    CE_koc = []
    for i,ce in enumerate (Name_ce_group):
        for ciso in ce:
            asorp_lh = 1 - alpha[i]/1000
            asorp_hh = asorp_lh**2
            Koc_ll = Koc_ce_ave[i]
            Koc_lh = Koc_ll*asorp_lh
            Koc_hh = Koc_ll*asorp_hh
            CE_koc.append(ciso)
        Koc_iso.append(Koc_ll)
        Koc_iso.append(Koc_lh)
        Koc_iso.append(Koc_hh)
    return CE_koc, Koc_iso

def set_diff(Name_ce_group,beta,diff_ce_ave,molar_masses):
    # set_diff provides the diffusion coefficients for each isotopologues
    # inputs are the difusion coefficients, average, for each CE (any temperature, no temp adjustments here)
    # molar masses for all CEs, fractionnation factors
    pd_list = [[],[]]
    for i,ce in enumerate(Name_ce_group):
        #m = mass_ave[i]
        d_av = diff_ce_ave[i]
        betaC = beta[i]
        for ciso in ce:
#            nb = len(ce)
            coef = 0
            for ciso_2 in ce:
                coef = coef + molar_masses.loc[ciso_2,'Abundance_0_pm']*(molar_masses.loc[ciso_2,'MW']/molar_masses.loc[ciso,'MW'])**(betaC)
            diff_y = d_av*coef*3600*24*365
            #print ciso, diff_y
            pd_list[0].append(ciso)
            pd_list[1].append(diff_y)
    return pd_list
"""
CE_C_koc, koc_C = func.set_koc(C_iso, A_C_SORP, Koc)
# CE_Cl_koc, koc_Cl = func.set_koc(Cl_iso, A_CL_SORP, Koc) NOT CREATED YET
# I created functions for calculating the diff coeff of each compound
CE_C_diff, diff_C = func.set_diff(C_iso,BETA_C_DIFF, diffav_0_pm_10, MW_abund_isotopologue )
CE_Cl_diff, diff_Cl = func.set_diff(Cl_iso,BETA_CL_DIFF, diffav_0_pm_10, MW_abund_isotopologue )

# Create a dataframe with all coefficients (so in the future it is possible to quickly check the info)
df_koc_C = pd.DataFrame(koc_C, index = CE_C_koc, columns = ['koc'])
df_koc_C.loc['Tracer','koc'] = inputs.loc['Tracer','Koc']
#data['R'] = foc_cl*data['koc']*rhob/porosity +1
df_diff_C.loc['Tracer','diff'] = inputs.loc['Tracer','diffav_0_pm_10']*3600*24*365

# Initial concentration
# Set all initial concentrations to 0
conc_init = pd.DataFrame(np.zeros(len(CE_iso)), index = CE_iso, columns = ['conc_init'])
conc0 = 4.4 # low conc = 0.001mM (9.7 mM just below TCE solubility) # For varying conc inputs
conc_init.loc['Tce_c_ll', 'conc_init'] = round(0.978551533*conc0,7)#(0.9778992136*conc0,7) # 6 max number of decimals (look in btn files to adjust)
conc_init.loc['Tce_c_lh', 'conc_init'] = round(0.021332208*conc0,7)#(0.0219773069*conc0,7)
conc_init.loc['Tce_c_hh', 'conc_init'] = round(0.000116259*conc0,7)#(0.0001234795*conc0,7)
conc_init.loc['Tracer', 'conc_init'] = round(conc0,2)#conc0,2)
data = pd.concat([df_diff_C, df_diff_Cl])
data = pd.concat([data,df_koc_C], axis = 1)

#####################################################
# Give to each coefficient a value
#####################################################
for i in np.arange(mcomp-2):  # Attribute concentrations sconc (BTN) and C (SSM package)
    exec "sconc%s=0" % (i+1)
    exec "C%s=%s" % (i+1,conc_init.loc[CE_iso[i],'conc_init'])
for i,ce_iso in enumerate(CE_iso): # Attributes diff, Kd, and initial concentrations
    exec "df%s=%s" % (i+1,data.loc[CE_iso[i],'diff']*porosity)#data.loc[CE_iso[i],'diff']*porosity)#data.loc['Tce_c_ll','diff']*porosity*3600*24*365)#data.loc[CE_iso[i],'diff']*porosity*3600*24*365)
    exec "Koc%s=%s" %   (i+1,data.loc[CE_iso[i],'koc'])#data.loc[CE_iso[i],'koc'])#data.loc['Tce_c_ll','koc'])#data.loc[CE_iso[i],'koc'])
#    exec "C%s=%s" %    (i+1,0)#data.loc[CE_iso[i],'conc_init'])
    data.loc[ce_iso,'numb'] = i + 1
for i,you_idiot in enumerate(np.ones(len(kin))):  # Attributes Cx (SSM package) of 1 for each kinetics. Not sure necessary
    exec "C%s=%s" %     (i + mcomp +1 ,you_idiot) # 
    exec "sconc%s=%s" % (i + mcomp +1 ,you_idiot)
for i,extra_conc in enumerate([7,14]): # Attributes Cx (SSM package) for pH, pe, and Rdoccurs
    exec "C%s=%s" %    (i+ len(CE_iso)+1, extra_conc) # 
    exec "sconc%s=%s" % (i+ len(CE_iso)+1 , extra_conc)    
# Manually change Rdoccurs #####################################################     Manually change Rdoccurs
#C25 = -1 # number = ncomp
sconc25  = Rdoccurs

In [ ]:
# Functions for printing a recap of all info
"""def save_text(path_flopy,filename,now,CE_iso,conc0,data,Lx,Ly,delr,delc,Vx,hg,perlen,nlay,al_sd,al_cl,trpv,trpt,porosity,foc_cl,foc_sd,rhob,th_cl,th_pl,A_C_SORP,BETA_C_DIFF,BETA_CL_DIFF):
    import os   
    os.chdir(path_flopy)  # ADD FOLDER WITH DATE AND TIME FOR UNIQUE FILES 
    toprint1 = []

    text_file = open(filename  + ".txt", "w")  # ADD DATE AND TIME FOR UNIQUE FILES
    text_file.write("###DESCRIPTION MODEL ")
    text_file.write("## Model Name:   %s \n" % (filename))
    toprint1.append(now)
    toprint1.append(CE_iso)
    toprint1.append("Initial concentration : %s" %(conc0))
    toprint1.append("GW velocity =  %s, head = %s, stress periods = %s" % (Vx, hg, perlen))
    toprint1.append("Lx =  %s, Ly = %s, nlay = %s" % (Lx, Ly, nlay))
    toprint1.append("Discretization: dx =  %s, dz = %s" % (delr, delc))
    toprint1.append("Dispersion Sand: aL =  %s, aT = %s, aV = %s" % (al_sd, trpt*al_sd, trpv*al_sd))
    toprint1.append("Dispersion Clay: aL =  %s, aT = %s, aV = %s" % (al_cl, trpt*al_cl, trpv*al_cl))
    toprint1.append("Porosity =  %s, foc clay = %s, foc sand = %s,rhob = %s" % (porosity, foc_cl, foc_sd, rhob))
    toprint1.append("TCE ll retardation in clay = " %(foc_cl*data.loc['Tce_c_ll','koc']*rhob/porosity))
    toprint1.append("Thickness clayer =  %s, H plume = %s," % (th_cl, th_pl))
    toprint1.append("Alpha sorption:  TCE = %s, DCE = %s, VC = %s, Eth = %s" % (A_C_SORP[0],A_C_SORP[1], A_C_SORP[2], A_C_SORP[3]))
    toprint1.append("Alpha diff C:  TCE = %s, DCE = %s, VC = %s, Eth = %s" % (BETA_C_DIFF[0],BETA_C_DIFF[1], BETA_C_DIFF[2], BETA_C_DIFF[3]))
    toprint1.append("Alpha diff CL: TCE = %s, DCE = %s, VC = %s, Eth = %s" % (BETA_CL_DIFF[0],BETA_CL_DIFF[1], BETA_CL_DIFF[2], BETA_CL_DIFF[3]))
    #toprint1.append("Input concentration TCE C: TCE_c_ll=  %s, TCE_c_lh =  %s, TCE_c_hh=  %s" % (data.loc['Tce_c_ll','conc_init'],data.loc['Tce_c_lh','conc_init'],data.loc['Tce_c_hh','conc_init']))
    #toprint1.append("Input concentration TCE Cl: TCE_lll =  %s, TCE_hll =  %s, TCE_hhl =  %s, TCE_hhh =  %s" % (data.loc['Tce_lll','conc_init'], data.loc['Tce_lhl','conc_init'], data.loc['Tce_lhh','conc_init'], data.loc['Tce_hhh','conc_init'] ))
    toprint1.append("###############################################")
    toprint1.append("Data are not necessarily all employed if the values are not set in the btn/ssm/disp/rct files!")
    toprint1.append("###############################################")
    toprint1.append(data)
    text_file.writelines("%s\n" % l for l in toprint1)
    text_file.close()

def save_for_plot(path_flopy,filename,now,CE_iso,conc0,s_d,Lx,Ly,delr,delc1,perlen,tsp,th_cl,th_pl,th_y1,nrow, ncol,x,y):
    import os   
    os.chdir(path_flopy)  # ADD FOLDER WITH DATE AND TIME FOR UNIQUE FILES 
    toprint1 = []
    toprint2 = []

    text_file = open("inputs_plot.txt", "w")  # ADD DATE AND TIME FOR UNIQUE FILES
    toprint1.append("DESCRIPTION MODEL")
    toprint1.append("ModelName   %s" % (filename))
    toprint1.append("C0 %s" %(conc0))
    toprint1.append("s_d %s" %(s_d))
    toprint1.append("Lx %s" % (Lx))
    toprint1.append("Ly %s" % (Ly))
    toprint1.append("nrow %s" % (nrow))
    toprint1.append("ncol %s" % (ncol))
    toprint1.append("delc1 %s" % (delc1))
    toprint1.append("delr %s" % (delr))
    toprint1.append("th_cl %s" % (th_cl))
    toprint1.append("th_y1 %s" % (th_y1))
    toprint1.append("th_pl %s" % (th_pl))
    toprint1.append("tsp %s" % (tsp))
    text_file.writelines("%s\n" % l for l in toprint1)
    text_file.close()

    text_file = open("inputs_plot_2.txt", "w")  # ADD DATE AND TIME FOR UNIQUE FILES
    text_file.write("\nModelName   %s \n" % (filename))
    toprint2.append(now)
    toprint2.append("x %s" % (x))
    toprint2.append("y %s" % (y))
    toprint2.append("perlen %s" % (perlen))

    toprint2.append(CE_iso)
    text_file.writelines("%s\n" % l for l in toprint2)
    text_file.close()"""

y,x,z = dis.get_node_coordinates() # Get nodes, very practical for plots
# Use function to create a file with more or less all infos. 
# Practical when plotting as well (for instance here the th_cl changes so it has to be adapted for some plots)
func.save_for_plot(path_flopy,filename,now,CE_iso,conc0,s_d,Lx,Ly,delr,delc1,perlen,tsp,th_cl,th_pl,th_y1,nrow,ncol,x,y)


In [ ]:
# Running PHT3D - can be done through cmd 

now = datetime.datetime.now()
import subprocess
import sys

os.chdir(path_flopy )
subprocess.call(['pht3dv210.exe', 'Z_run_pht3d.nam'])



In [ ]:
#Additional infos

# Get run duration
import datetime
now = datetime.datetime.now()
now_end = datetime.datetime.now() 
print (now - now_end)

# Add a beeping sound when run is completed so you stop doing whatever you were doing and look at your results
import winsound
frequency = 500  # Set Frequency To 2500 Hertz
duration = 1000  # Set Duration To 1000 ms == 1 second

winsound.Beep(frequency, duration)
winsound.Beep(frequency*2, duration/2)
winsound.Beep(frequency, duration)



In [ ]:
# PLOT


sim_list = ['rate_0','e_C_0','aS1D1']#,'foc_sd_1']
for var in sim_list:
        exec "%s=[]" %(var) 
df_list  = [rate_0,e_C_0,aS1D1]#,foc_sd_1]

for (filename,df_sim) in zip(sim_list,df_list):

        path_plot =  'D:\\0_Recent_files\\8_Model_flopy\\6_Model_final\\2_Variations' #5_Model_final\\2_P3_Variations'#6_Model_final\\2_Variations' #5_Model_final\\2_P3_Variations'#
        path_flopy = path_plot + '\\Results\\' + filename
        ##################################################################################################################
        # model specific parameters
        # All of this because th_cl is not the same for all models
        ##################################################################################################################
        model_inputs = pd.read_csv(path_flopy + '\\inputs_plot.txt', delimiter = ' ', index_col = 0 , header = 1)
        for param in ['s_d','th_cl']:
            exec "%s=%s" %(param,model_inputs.ix[param,0] ) 
        delc1 = 0.05; th_small_disc1 = th_y1 + th_cl 
        delc2 = 0.1; th_large_disc2 = th_pl + 1.5 
        delc3 = 0.25; th_large_disc3 = Ly - th_small_disc1 - th_large_disc2
        delc = np.hstack(((delc1*np.ones(th_small_disc1/delc1)),delc2*np.ones(th_large_disc2/delc2),delc3*np.ones(th_large_disc3/delc3)))#
        nrow = len(delc)
        nrowbott = int(nrow - th_y1/delc1)
        nrowtop = int(nrow - (th_y1 + th_cl)/delc1)
        #PLUME # 2 m thick plume above the clay layer
        #This model implies the plume bottom is at the surface of the aquitard
        nrow_pl_top  = int(nrow - (th_y1 + th_cl)/delc1 - (th_pl)/delc2)
        nrow_pl_bott = int(nrow - (th_y1 + th_cl)/delc1)
        
        mfexe = 'C:\\Program Files (x86)\\PM8\\modflow2005\\mf2005'
        mf = flopy.modflow.Modflow(filename, exe_name= mfexe,  laytyp=0)
        dis = flopy.modflow.ModflowDis(mf, nlay, nrow, ncol, nper = len(perlen),
                                       delr=delr, delc=delc,
                                       top=ztop, botm=botm[1:],
                                       perlen = perlen, nstp = nstp)
        y,x,z = dis.get_node_coordinates() # Get nodes
        y = y - th_cl - th_y1
        ##################################################################################################################
        # Import all data
        for i,ce in enumerate(Master_species[0:9]):
            exec "%s_ucn = flopy.utils.binaryfile.UcnFile(path_flopy + '\%s00%s.UCN')" % (ce,plot_typ,i+1)
            exec "%s = np.NaN*np.zeros((len(perlen),nrow,ncol))" % (ce)
            for tper in np.arange(0,len(perlen)):
                exec "%s[tper] = %s_ucn.get_data(kstpkper =(int(perlen[tper]/tsp-1),tper) )" % (ce,ce)#(int(perlen[tper]/tsp-1),tper) )
        for i,ce in enumerate(Master_species[9:]):
            exec "%s_ucn = flopy.utils.binaryfile.UcnFile(path_flopy + '\%s0%s.UCN')" % (ce,plot_typ,i+10)
            exec "%s = np.NaN*np.zeros((len(perlen),nrow,ncol))" % (ce)
            for tper in np.arange(0,len(perlen)):
                exec "%s[tper] = %s_ucn.get_data(kstpkper =(int(perlen[tper]/tsp-1),tper))" % (ce,ce)#(kstpkper =(int(perlen[tper]/tsp-1),tper) )
        times = Tce_c_ll_ucn.get_times()
        TCE = Tce_c_ll+Tce_c_lh+Tce_c_hh
        DCE =  Dce_c_ll+Dce_c_lh+Dce_c_hh
        VC = Vc_c_ll+Vc_c_lh+Vc_c_hh
        Eth =  Eth_c_ll+Eth_c_lh+Eth_c_hh
        TCE_cimb = Tce_c_ll+Tce_c_lh+Tce_c_hh
        DCE_cimb =  Dce_c_ll+Dce_c_lh+Dce_c_hh
        VC_cimb = Vc_c_ll+Vc_c_lh+Vc_c_hh
        Eth_cimb =  Eth_c_ll+Eth_c_lh+Eth_c_hh
        Tce_c_ll[TCE < dlim] = np.NaN
        Tce_c_lh[TCE < dlim] = np.NaN
        Tce_c_hh[TCE < dlim] = np.NaN
        Dce_c_ll[DCE < dlim] = np.NaN
        Dce_c_lh[DCE < dlim] = np.NaN
        Dce_c_hh[DCE < dlim] = np.NaN
        Vc_c_ll[VC < dlim] = np.NaN
        Vc_c_lh[VC < dlim] = np.NaN
        Vc_c_hh[VC < dlim] = np.NaN
        Eth_c_ll[Eth < dlim] = np.NaN
        Eth_c_lh[Eth < dlim] = np.NaN
        Eth_c_hh[Eth < dlim] = np.NaN
        TCEdC = 1000*((((Tce_c_lh+2*Tce_c_hh)/(Tce_c_lh+2*Tce_c_ll))/0.011237)-1)
        DCEdC = 1000*((((Dce_c_lh+2*Dce_c_hh)/(Dce_c_lh+2*Dce_c_ll))/0.011237)-1)
        VCdC = 1000*((((Vc_c_lh+2*Vc_c_hh)/(Vc_c_lh+2*Vc_c_ll))/0.011237)-1)
        EthdC = 1000*((((Eth_c_lh+2*Eth_c_hh)/(Eth_c_lh+2*Eth_c_ll))/0.011237)-1)
        TCEdC_cimb = 1000*((((Tce_c_lh+2*Tce_c_hh)/(Tce_c_lh+2*Tce_c_ll))/0.011237)-1)
        DCEdC_cimb = 1000*((((Dce_c_lh+2*Dce_c_hh)/(Dce_c_lh+2*Dce_c_ll))/0.011237)-1)
        VCdC_cimb = 1000*((((Vc_c_lh+2*Vc_c_hh)/(Vc_c_lh+2*Vc_c_ll))/0.011237)-1)
        EthdC_cimb = 1000*((((Eth_c_lh+2*Eth_c_hh)/(Eth_c_lh+2*Eth_c_ll))/0.011237)-1)
        # concentrations below
        #TCE[TCE < dlim] = np.NaN
        #DCE[DCE < dlim] = np.NaN
        #VC[VC < dlim] = np.NaN
        #Eth[Eth < dlim] = np.NaN
        MTCE = 131.; MDCE = 97.; MVC = 62.; METH = 28.
        #dlim_TCE = 10**-6/MTCE;dlim_DCE = 10**-6/MDCE;dlim_VC = 10**-6/MVC;dlim_ETH = 10**-6/METH
        #dlim_TCE = 10**-6/MTCE;dlim_DCE = 10**-6/MDCE;dlim_VC = 10**-6/MVC;dlim_ETH = 10**-6/METH
        TCE_m = TCE * MTCE*1000.;DCE_m = DCE * MDCE*1000.;VC_m = VC * MVC*1000.;Eth_m = Eth * METH*1000.;Tracer_m = Tracer*MTCE*1000.

        for ce,ce_m in zip([TCE,DCE,VC,Eth],[TCE_m,DCE_m,VC_m,Eth_m]):
            ce[ce_m < 1] = np.NaN
        for ce,cedC,ce_m in zip([TCE,DCE,VC,Eth],[TCEdC,DCEdC,VCdC,EthdC],[TCE_m,DCE_m,VC_m,Eth_m]):
            cedC[ce_m < 1] = np.NaN
        for CE,CEdC,ce_m in zip([TCE_cimb,DCE_cimb,VC_cimb,Eth_cimb],[TCEdC_cimb,DCEdC_cimb,VCdC_cimb,EthdC_cimb],[TCE_m,DCE_m,VC_m,Eth_m]):
            CE[ce_m < 1 ] = 0
            CEdC[ce_m < 1 ] = 0
        for ce in [TCE_m,DCE_m,VC_m,Eth_m,Tracer_m]:
            ce[ce < 1] = np.NaN
        i=0
        Ctot = TCE + DCE + VC + Eth
        Ctot_cimb =  TCE_cimb + DCE_cimb + VC_cimb + Eth_cimb
        CEtot_cimb =  TCE_cimb + DCE_cimb + VC_cimb 
        CIMB = (TCE_cimb*TCEdC_cimb + DCE_cimb*DCEdC_cimb + VC_cimb*VCdC_cimb + Eth_cimb*EthdC_cimb) / Ctot_cimb          
